In [ ]:
# Application of FL task
from MLModel import *
from utils import *

import torch
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics

from scipy.optimize import curve_fit
from scipy.stats import norm, rv_histogram, entropy, wasserstein_distance

import os
import copy
import warnings
warnings.filterwarnings("ignore")

#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#export CUDA_VISIBLE_DEVICES=""

device = torch.device("cpu")
print(device)

In [ ]:
num_client = 4
x=[]
y=[]
data_pth = 'winequality-red/'
feature = 'alcohol'

output_class = 6

def local_data(data_pth, feature, kid):
    # test set
    x=[]
    y=[]
    for i in range(1, num_client+1):
        x.append(np.load("model/"+data_pth+feature+"/c"+str(kid)+"/test_x"+str(i)+".npy"))
        y.append(np.load("model/"+data_pth+feature+"/c"+str(kid)+"/test_y"+str(i)+".npy"))

    non_iid = []
    real_x = []
    real_y = []
    for i in range(1, num_client+1):
        real_x.append(np.load("model/"+data_pth+feature+"/c"+str(kid)+"/train_x"+str(i)+".npy"))
        real_y.append(np.load("model/"+data_pth+feature+"/c"+str(kid)+"/train_y"+str(i)+".npy"))

    global_x = np.concatenate((x[0], x[1]))
    global_x = np.concatenate((global_x, x[2]))
    global_x = np.concatenate((global_x, x[3]))

    global_y = np.concatenate((y[0], y[1]))
    global_y = np.concatenate((global_y, y[2]))
    global_y = np.concatenate((global_y, y[3]))
    #print(global_x.shape)
    #print(global_y.shape)
    num_feature = global_x.shape[1]
    
    dist_info = []
    discrete_feature = []
    for i in range(num_feature):
        # discrete?
        unique_feature = np.unique(global_x[:, i], return_counts=True)
        if len(unique_feature[0]) < 25:
            # compute probability (frequence) of each element
            dist_info.append(tuple((unique_feature[0], unique_feature[1]/unique_feature[1].sum())))
            discrete_feature.append(i)
        else:
            # compute mean and std
            mu = np.mean(global_x[:, i])
            sigma = np.std(global_x[:, i])
            dist_info.append(tuple((mu, sigma)))
        
    return real_x, real_y, global_x, global_y, dist_info

In [ ]:
def load_model_dpsgd(eps, i, kid):
    global_iter = 5
    w_ml = []
        
    m0 = model(num_feature, output_class).to(device)
    m0.load_state_dict(torch.load('model/'+data_pth+feature+'/c'+str(kid)+"/"+str(i)+"/"+model.__name__+'/init/eps'+str(eps)+'/init.pth', map_location=torch.device('cpu')))
    param_name = list(m0.state_dict().keys())
        
    mg = model(num_feature, output_class).to(device)
    mg.load_state_dict(torch.load('model/'+data_pth+feature+'/c'+str(kid)+"/"+str(i)+'/'+model.__name__+'/epoch5'+'/eps'+str(eps)+'/global.pth', map_location=torch.device('cpu')))
    mg.eval()
    mg_params = torch.cat([torch.flatten(mg.state_dict()[name]) for name in param_name], -1)
            
    for client_id in range(num_client):
        model_par = []

        # compute weights
        ml_params = []
        ml = []

        delta_m_local = []
        w_list = []

        for epoch in range(1,global_iter+1):
            # load local models
            m0 = model(num_feature, output_class).to(device)
            m0.load_state_dict(torch.load('model/'+data_pth+feature+'/c'+str(kid)+"/"+str(i)+'/'+model.__name__+'/epoch'+str(epoch)+'/eps'+str(eps)+'/'+str(client_id)+'.pth', 
                                           map_location=torch.device('cpu')))
            m0.eval()
            m0_params = torch.cat([torch.flatten(m0.state_dict()[name]) for name in param_name], -1)
            ml.append(copy.deepcopy(m0.state_dict()))
            
            delta_m_local = torch.norm(m0_params - mg_params, p=2)
            w_list.append(delta_m_local.item())
            
        w_list = torch.softmax(torch.Tensor(w_list), dim=0)

        new_par = copy.deepcopy(m0.state_dict())
        for name in new_par:
            new_par[name] = torch.zeros(new_par[name].shape)
        for idx, par in enumerate(ml):
            for name in new_par:
                new_par[name] += par[name]*w_list[idx]
        
        m0 = model(num_feature, output_class).to(device)
        m0.load_state_dict(copy.deepcopy(new_par))
        m0.eval()
        w_ml.append(copy.deepcopy(m0))

    g = model(num_feature, output_class).to(device)
    g.load_state_dict(torch.load('model/'+data_pth+feature+'/c'+str(kid)+"/"+str(i)+'/'+model.__name__+'/epoch5'+'/eps'+str(eps)+'/global.pth', map_location=torch.device('cpu')))
    g.eval()
    
    m0 = model(num_feature, output_class).to(device)
    m0.load_state_dict(torch.load('model/'+data_pth+feature+'/c'+str(kid)+"/"+str(i)+"/"+model.__name__+'/init/eps'+str(eps)+'/init.pth', map_location=torch.device('cpu')))
    m0.eval()
    return (w_ml, g, m0)

In [ ]:
def PGD_attack(c, feature_dis, num_feature, ml, g, lr, var_factor):
    r_min = 0
    r_max = 1
    x = torch.tensor(np.random.uniform(r_min, r_max, num_feature)).float().requires_grad_()
    opt = optim.Adam([x], lr=lr)

    for _ in range(25):
        yg = torch.softmax(g(x),0)
        yc = torch.softmax(ml[c](x),0)
        
        loss = torch.nn.MSELoss()(yg, yc) + var_factor*x.var()

        opt.zero_grad()
        loss.backward()
        opt.step()
        
        # projection
        x = torch.clamp(x, r_min, r_max).clone().detach().numpy()
        for f in range(num_feature):
            # discrete data
            if type(feature_dis[f][0]) is np.ndarray:
                x[f] = np.random.choice(feature_dis[f][0], 1)[0]
        x = torch.tensor(x).requires_grad_()
        opt = optim.Adam([x], lr=lr)
    return x.detach().numpy()


def gen_synthesis_PGD(dist_info, num_feature, num_client, fake_sample_size, ml, g, lr, var_factor):
    fake_sample_size = int(fake_sample_size / num_client)
    sample_list = []
    for c in range(num_client):
        tmpsample = []
        for i in range(fake_sample_size):
            x = PGD_attack(c, dist_info, num_feature, ml, g, lr, var_factor)
            tmpsample.append(x)
            if len(tmpsample)%400 == 0:
                print(len(tmpsample))
        sample_list.append(copy.deepcopy(np.array(tmpsample)))
    return sample_list


def PGD_noised_sample(feature_dis, num_feature, num_client, fake_sample_size, ml, g, m0, lr, var_factor):
    fake_sample_size = int(fake_sample_size / num_client)
    sample_list = []
    sample_sim_list = []
    sample_noise_lsit = []
    sample_simnoise_lise = []
    subsample_list = []
    
    for c in range(num_client):
        pgd_sample = []
        pgd_noise = []

        d_max = -1
        d_min = 9999

        cnt = 0
        d_list = []
        d_threshold  = 0
        
        while len(pgd_noise) < fake_sample_size:
            x = PGD_attack(c, feature_dis, num_feature, ml, g, lr, var_factor)
            if len(pgd_sample) < fake_sample_size:
                pgd_sample.append(copy.deepcopy(x))
                pgd_noise.append(copy.deepcopy(x))

        sample_list.append(copy.deepcopy(np.array(pgd_sample)))
        sample_noise_lsit.append(copy.deepcopy(np.array(pgd_noise)))
    return (sample_list, sample_noise_lsit)



def hc_noise_attack(feature_dis, num_feature, num_client, fake_sample_size, ml, g, prior_dist):
    fake_sample_size = int(fake_sample_size / num_client)
    sample_list = []
    subsample_list = []
    
    for c in range(num_client):
        hc_sample = []
        hc_subsample = []
        threshold = 1e-3
        
        d_max = -1
        d_min = 9999
        
        cnt = 0
        d_list = []
        d_threshold  = 0
        while len(hc_subsample) < fake_sample_size:
            x = hc_attack(c, feature_dis, num_feature, ml, g, prior_dist, threshold)
            j = 0
            
            # generate a valid sample
            while x is None:
                x = hc_attack(c, feature_dis, num_feature, ml, g, prior_dist, threshold)
                j += 1
                if j > 10:
                    threshold *= 2
                    #print("threshold =", threshold)
                    j = 0
                    
            if len(hc_sample) < fake_sample_size:
                hc_sample.append(copy.deepcopy(x))
                hc_subsample.append(copy.deepcopy(x))
            
        sample_list.append(copy.deepcopy(np.array(hc_sample)))
        subsample_list.append(copy.deepcopy(np.array(hc_subsample)))
    return (sample_list, subsample_list)

## Attack LR

In [ ]:
att_method = 'pgd'
model = LogisticRegression
# model = LightMLP2

if hasattr(model, '__name__'):
    model_name = model.__name__ 

for eps in [0.25, 0.5, 1, 2, 4, 8, 1e12]:
    rg_uni = []
    rg_gua = []
    
    pgd_att = []
    # hc_att = []
    
    for kid in range(1, 6):
        real_x, real_y, global_x, global_y, dist_info = local_data(data_pth, feature, kid)
        num_feature = global_x.shape[1]
        for _ in range(1, 6):
            ml, g, m0 = load_model_dpsgd(eps, _, kid)
            fake_sample_size = 4000

            if att_method == 'pgd':
                sample_list, pgd_noise = PGD_noised_sample(dist_info, num_feature, num_client, fake_sample_size, ml, g, m0, 0.1, 0.1)
                np.save('model/'+data_pth+feature+'/c'+str(kid)+'/'+str(_)+'/'+model_name+"/pgd_sample_eps"+str(eps)+".npy", np.array(sample_list))
                np.save('model/'+data_pth+feature+'/c'+str(kid)+'/'+str(_)+'/'+model_name+"/pgd_noise_sample_eps"+str(eps)+".npy", np.array(pgd_noise))
                
            elif att_method == 'hc':
                sample_list, pgd_noise = hc_noise_attack(dist_info, num_feature, num_client, fake_sample_size, ml, g, 'uniform')
                np.save('model/'+data_pth+feature+'/c'+str(kid)+'/'+str(_)+'/'+model_name+"/hc_sample_eps"+str(eps)+".npy", np.array(sample_list))
                np.save('model/'+data_pth+feature+'/c'+str(kid)+'/'+str(_)+'/'+model_name+"/hc_noise_sample_eps"+str(eps)+".npy", np.array(pgd_noise))

            r_min = 0
            r_max = 1

            gaussian_est = [np.array(synthetic_gaussian(dist_info, num_feature, int(fake_sample_size/4))) for _ in range(4)]
            uniform_est = [np.array(synthetic_uniform(dist_info, num_feature, int(fake_sample_size/4))) for _ in range(4)]

            kl_baseline_g = []
            kl_baseline_u = []

            kl_sample = []
            kl_sample_noise = []

            per_feature = []
            per_baseline = []
            for n_feature in range(num_feature):
                for k in range(0,4):
                    kl_baseline_g.append(wasserstein_distance(gaussian_est[k][:,n_feature], real_x[k][:,n_feature]))
                    kl_baseline_u.append(wasserstein_distance(uniform_est[k][:,n_feature], real_x[k][:,n_feature]))

                    kl_sample.append(wasserstein_distance(sample_list[k][:,n_feature], real_x[k][:,n_feature]))
                    kl_sample_noise.append(wasserstein_distance(pgd_noise[k][:,n_feature], real_x[k][:,n_feature]))
                per_feature.append(np.mean(kl_sample[-4:]))
                
            rg_uni.append(np.mean(kl_baseline_u))
            rg_gua.append(np.mean(kl_baseline_g))
            pgd_att.append(np.mean(kl_sample))
    
    print("eps =", eps)
    print("baseline uniform: ", np.mean(rg_uni))
    print("baseline gaussian:", np.mean(rg_gua))
    
    print("attack: ", np.mean(pgd_att))
    print("=======")

## Black-Box LR

In [ ]:
att_method = 'hc'
model = LogisticRegression
# model = LightMLP2

if hasattr(model, '__name__'):
    model_name = model.__name__ 

for eps in [0.25, 0.5, 1, 2, 4, 8, 1e12]:
    rg_uni = []
    rg_gua = []
    
    pgd_att = []
    # hc_att = []
    
    for kid in range(1, 6):
        real_x, real_y, global_x, global_y, dist_info = local_data(data_pth, feature, kid)
        num_feature = global_x.shape[1]
        for _ in range(1, 6):
            ml, g, m0 = load_model_dpsgd(eps, _, kid)
            fake_sample_size = 4000

            if att_method == 'pgd':
                sample_list, pgd_noise = PGD_noised_sample(dist_info, num_feature, num_client, fake_sample_size, ml, g, m0, 0.1, 0.1)
                np.save('model/'+data_pth+feature+'/c'+str(kid)+'/'+str(_)+'/'+model_name+"/pgd_sample_eps"+str(eps)+".npy", np.array(sample_list))
                np.save('model/'+data_pth+feature+'/c'+str(kid)+'/'+str(_)+'/'+model_name+"/pgd_noise_sample_eps"+str(eps)+".npy", np.array(pgd_noise))
                
            elif att_method == 'hc':
                sample_list, pgd_noise = hc_noise_attack(dist_info, num_feature, num_client, fake_sample_size, ml, g, 'uniform')
                np.save('model/'+data_pth+feature+'/c'+str(kid)+'/'+str(_)+'/'+model_name+"/hc_sample_eps"+str(eps)+".npy", np.array(sample_list))
                np.save('model/'+data_pth+feature+'/c'+str(kid)+'/'+str(_)+'/'+model_name+"/hc_noise_sample_eps"+str(eps)+".npy", np.array(pgd_noise))

            r_min = 0
            r_max = 1

            gaussian_est = [np.array(synthetic_gaussian(dist_info, num_feature, int(fake_sample_size/4))) for _ in range(4)]
            uniform_est = [np.array(synthetic_uniform(dist_info, num_feature, int(fake_sample_size/4))) for _ in range(4)]

            kl_baseline_g = []
            kl_baseline_u = []

            kl_sample = []
            kl_sample_noise = []

            per_feature = []
            per_baseline = []
            for n_feature in range(num_feature):
                for k in range(0,4):
                    kl_baseline_g.append(wasserstein_distance(gaussian_est[k][:,n_feature], real_x[k][:,n_feature]))
                    kl_baseline_u.append(wasserstein_distance(uniform_est[k][:,n_feature], real_x[k][:,n_feature]))

                    kl_sample.append(wasserstein_distance(sample_list[k][:,n_feature], real_x[k][:,n_feature]))
                    kl_sample_noise.append(wasserstein_distance(pgd_noise[k][:,n_feature], real_x[k][:,n_feature]))
                per_feature.append(np.mean(kl_sample[-4:]))
                
            rg_uni.append(np.mean(kl_baseline_u))
            rg_gua.append(np.mean(kl_baseline_g))
            pgd_att.append(np.mean(kl_sample))
    
    print("eps =", eps)
    print("baseline uniform: ", np.mean(rg_uni))
    print("baseline gaussian:", np.mean(rg_gua))
    
    print("attack: ", np.mean(pgd_att))
    print("=======")

## PGDA NN

In [ ]:
att_method = 'pgd'
# model = LogisticRegression
model = LightMLP2

if hasattr(model, '__name__'):
    model_name = model.__name__ 

for eps in [0.25, 0.5, 1, 2, 4, 8, 1e12]:
    rg_uni = []
    rg_gua = []
    
    pgd_att = []
    # hc_att = []
    
    for kid in range(1, 6):
        real_x, real_y, global_x, global_y, dist_info = local_data(data_pth, feature, kid)
        num_feature = global_x.shape[1]
        for _ in range(1, 6):
            ml, g, m0 = load_model_dpsgd(eps, _, kid)
            fake_sample_size = 4000

            if att_method == 'pgd':
                sample_list, pgd_noise = PGD_noised_sample(dist_info, num_feature, num_client, fake_sample_size, ml, g, m0, 0.1, 0.1)
                np.save('model/'+data_pth+feature+'/c'+str(kid)+'/'+str(_)+'/'+model_name+"/pgd_sample_eps"+str(eps)+".npy", np.array(sample_list))
                np.save('model/'+data_pth+feature+'/c'+str(kid)+'/'+str(_)+'/'+model_name+"/pgd_noise_sample_eps"+str(eps)+".npy", np.array(pgd_noise))
                
            elif att_method == 'hc':
                sample_list, pgd_noise = hc_noise_attack(dist_info, num_feature, num_client, fake_sample_size, ml, g, 'uniform')
                np.save('model/'+data_pth+feature+'/c'+str(kid)+'/'+str(_)+'/'+model_name+"/hc_sample_eps"+str(eps)+".npy", np.array(sample_list))
                np.save('model/'+data_pth+feature+'/c'+str(kid)+'/'+str(_)+'/'+model_name+"/hc_noise_sample_eps"+str(eps)+".npy", np.array(pgd_noise))

            r_min = 0
            r_max = 1

            gaussian_est = [np.array(synthetic_gaussian(dist_info, num_feature, int(fake_sample_size/4))) for _ in range(4)]
            uniform_est = [np.array(synthetic_uniform(dist_info, num_feature, int(fake_sample_size/4))) for _ in range(4)]

            kl_baseline_g = []
            kl_baseline_u = []

            kl_sample = []
            kl_sample_noise = []

            per_feature = []
            per_baseline = []
            for n_feature in range(num_feature):
                for k in range(0,4):
                    kl_baseline_g.append(wasserstein_distance(gaussian_est[k][:,n_feature], real_x[k][:,n_feature]))
                    kl_baseline_u.append(wasserstein_distance(uniform_est[k][:,n_feature], real_x[k][:,n_feature]))

                    kl_sample.append(wasserstein_distance(sample_list[k][:,n_feature], real_x[k][:,n_feature]))
                    kl_sample_noise.append(wasserstein_distance(pgd_noise[k][:,n_feature], real_x[k][:,n_feature]))
                per_feature.append(np.mean(kl_sample[-4:]))
                
            rg_uni.append(np.mean(kl_baseline_u))
            rg_gua.append(np.mean(kl_baseline_g))
            pgd_att.append(np.mean(kl_sample))
    
    print("eps =", eps)
    print("baseline uniform: ", np.mean(rg_uni))
    print("baseline gaussian:", np.mean(rg_gua))
    
    print("attack: ", np.mean(pgd_att))
    print("=======")

## Attack Label Distribution

In [ ]:
def naive_label_attack(client_num, sample, ml, g):
    predit_label = [[] for _ in range(client_num)]
    predit_prob = [[] for _ in range(client_num)]
    # random_guess = [[np.random.choice(list(range(output_class))) for _ in range(1250)] for _ in range(client_num)]
    random_guess = []
    for c in range(client_num):
        for x in sample[c]:
            pred_g = torch.softmax(ml[c](torch.tensor(x).float()),0)
                
            pred_label = torch.max(pred_g, 0)[1].item()
            predit_label[c].append(pred_label)
            for i in range(output_class):
                for _ in range(100):
                    if np.random.random() < pred_g[i]:
                        predit_prob[c].append(i)
                    
        for i in range(output_class):
            predit_label[c].append(i)
            predit_prob[c].append(i)
            
    # generate grandom guess results
    for k in range(100):
        random_guess_curr = []
        for c in range(client_num):
            rv = [np.random.random() for _ in range(output_class)]
            p = rv / np.sum(rv)
            curr_l = []
            for l in range(output_class):
                curr_l += [l for _ in range(int(2000*p[l]))]
            random_guess_curr.append(curr_l)
        random_guess.append(random_guess_curr)
    return (predit_label, predit_prob, random_guess)

In [ ]:
model = LightMLP2
#model = LogisticRegression
#att_method = 'hc'
att_method = 'pgda'

if hasattr(model, '__name__'):
    model_name = model.__name__
    
real_x, real_y, global_x, global_y, dist_info = local_data(data_pth, feature, 1)
num_feature = global_x.shape[1]

att_res = []

for eps in [0.25, 0.5, 1, 2, 4, 8, 1e12]:
    kl_sample = []
    kl_sample_noise = []
    kl_baseline_u = []

    for kid in range(1,6):
        for _ in range(1,6):
            ml, g, m0 = load_model_dpsgd(eps, _, kid)

            if att_method == 'hc':
                sample_list = np.load('model/'+data_pth+feature+'/c'+str(kid)+'/'+str(_)+'/'+model_name+"/hc_sample_eps"+str(eps)+".npy")
            else:
                sample_list = np.load('model/'+data_pth+feature+'/c'+str(kid)+'/'+str(_)+'/'+model_name+"/pgd_sample_eps"+str(eps)+".npy")

            #print(sample_list[3].shape)

            predit_label, predit_prob, random_guess = naive_label_attack(num_client, sample_list, ml, g)

            per_feature = []
            per_baseline = []
            for k in range(0,4):
                kl_sample.append(wasserstein_distance(predit_label[k], real_y[k]))
                kl_sample_noise.append(wasserstein_distance(predit_prob[k], real_y[k]))

            # baseline
            for k in range(0,4):
                for t in range(np.shape(random_guess)[0]):
                    kl_baseline_u.append(wasserstein_distance(random_guess[t][k], real_y[k]))

    att_res.append(np.mean(kl_sample_noise))
    print("eps =", eps)
    print("baseline uniform: ", np.mean(kl_baseline_u))
    
    print("attack g predit: ", np.mean(kl_sample))
    print("attack pred prob:", np.mean(kl_sample_noise))
    print("=======")
print(att_res)